In [15]:
import asyncio
from ldp.agent import SimpleAgent
import nest_asyncio
from aviary.core import Message, Tool,FunctionInfo

from pydantic import BaseModel, Field

In [16]:
nest_asyncio.apply()

In [ ]:
# Define input/output types for the calculator
class CalculatorInput(BaseModel):
    x: float = Field(description="First number")
    y: float = Field(description="Second number")

class CalculatorOutput(BaseModel):
    result: float = Field(description="The sum of x and y")


In [ ]:
async def test_simple_agent():
    # Define the calculator function
    async def calculator(input_data: CalculatorInput) -> CalculatorOutput:
        return CalculatorOutput(result=input_data.x + input_data.y)

    # Create the tool properly structured
    function_info = FunctionInfo(
        name="add",
        description="Add two numbers together",
        parameters=CalculatorInput.model_json_schema(),
        function=calculator,
        input_type=CalculatorInput,
        output_type=CalculatorOutput
    )

    calculator_tool = Tool(
        name="calculator",
        description="A simple calculator that can add two numbers",
        function=function_info
    )

    # Initialize the agent with the tool
    agent = SimpleAgent(
        llm_model={
            "model": "gpt-3.5-turbo",
            "temperature": 0.1
        },
        sys_prompt="You are a helpful assistant that can perform calculations."
    )

    # Initialize agent state with tools
    state = await agent.init_state(tools=[calculator_tool])

    # Create a test message
    test_message = Message(
        role="user",
        content="Can you help me add 5 and 3?"
    )

    # Get agent's action and new state
    action, new_state, cost = await agent.get_asv(state, [test_message])
    
    # Print results
    print("Action:", action.value.content)
    print("Cost:", cost)
    print("Total tokens used:", new_state.total_tokens)
    print("Total cost so far:", new_state.total_cost)

    return action, new_state, cost


In [21]:
# For Jupyter environment:
await test_simple_agent()


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



/Users/apple/opt/anaconda3/envs/paperqa_env/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'test_simple_agent' was never awaited
  return compile(source, filename, mode, flags,


NotFoundError: litellm.NotFoundError: OpenAIException - Error code: 403 - {'error': {'message': 'Project `proj_8WO6cHG6Ics15ycd35qOE2RY` does not have access to model `gpt-4`', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}